<a href="https://colab.research.google.com/github/EmperoR1127/CSI5155_project/blob/master/csi5155_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
from scipy.io import arff
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "/content/drive/My Drive/Images/"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

data = arff.loadarff('/content/drive/My Drive/Data/seismic-bumps.arff')
seismic_bumps_data = pd.DataFrame(data[0])
#pre-process the train_set
train_labels = seismic_bumps_data[["class"]].copy()
train_set = seismic_bumps_data.drop(["class"], axis=1)
train_labels["class"] = train_labels["class"].map(lambda x: str(x)[2])
train_set_num = seismic_bumps_data.drop(["seismic","seismoacoustic","shift", "ghazard", "class"], axis=1)
train_set_cat = seismic_bumps_data.drop(["genergy","gpuls","gdenergy", "gdpuls", "nbumps", "nbumps2", "nbumps3", "nbumps4", "nbumps5", "nbumps6", "nbumps7", "nbumps89", "energy", "maxenergy", "class"], axis=1)
#build the pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),])
full_pipeline = ColumnTransformer([("num", num_pipeline, list(train_set_num)),("cat", OneHotEncoder(), list(train_set_cat)),])
#prepare the data
train_set_prepared = full_pipeline.fit_transform(train_set)
#prepare the target
encoder = LabelEncoder()
train_labels_prepared = encoder.fit_transform(train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [3]:
#rebalance the dataset using 3 different approaches
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
from imblearn.combine import SMOTEENN
#rebalance the dataset using oversampling (random oversampling)
ros = RandomOverSampler(random_state=42)
ros_train_set_prepared, ros_train_labels_prepared = ros.fit_resample(train_set_prepared, train_labels_prepared)
print("Class distribution of oversampling" + str(sorted(Counter(ros_train_labels_prepared).items())))

#rebalance the dataset using undersampling (nearest neightbours)
renn = RepeatedEditedNearestNeighbours()
renn_train_set_prepared, renn_train_labels_prepared = renn.fit_resample(train_set_prepared, train_labels_prepared)
print("Class distribution of undersampling" + str(sorted(Counter(renn_train_labels_prepared).items())))

#rebalance the dataset using balanced sampling (SMOTEENN)
smote_enn = SMOTEENN(random_state=0)
smote_enn_train_set_prepared, smote_enn_train_labels_prepared = smote_enn.fit_resample(train_set_prepared, train_labels_prepared)
print("Class distribution of balanced sampling" + str(sorted(Counter(smote_enn_train_labels_prepared).items())))

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


Class distribution of oversampling[(0, 2414), (1, 2414)]
Class distribution of undersampling[(0, 1920), (1, 170)]
Class distribution of balanced sampling[(0, 1818), (1, 2194)]


In [4]:
#train the model using DECISION TREE algorithm with 10 fold cross validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score
dt_clf = DecisionTreeClassifier()
#train the model on the oversampling dataset
dt_ros_train_prediction = cross_val_predict(dt_clf.fit(ros_train_set_prepared, ros_train_labels_prepared), ros_train_set_prepared, ros_train_labels_prepared, cv=10)
#train the model on the undersampling dataset
dt_renn_train_prediction = cross_val_predict(dt_clf.fit(renn_train_set_prepared, renn_train_labels_prepared), renn_train_set_prepared, renn_train_labels_prepared, cv=10)
#train the model on the balanced sampling dataset
dt_smote_enn_train_prediction = cross_val_predict(dt_clf.fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared), smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10)

#calculate the presion and recall of oversampling dataset
dt_ros_precision_score = precision_score(ros_train_labels_prepared, dt_ros_train_prediction)
dt_ros_recall_score = recall_score(ros_train_labels_prepared, dt_ros_train_prediction)
print("precision of decision tree model on oversampling dataset is + %f" % dt_ros_precision_score)
print("recall of decision tree model on oversampling dataset is + %f" % dt_ros_recall_score)
print("-----------------------------------------")

#calculate the presion and recall of undersampling dataset
dt_renn_precision_score = precision_score(renn_train_labels_prepared, dt_renn_train_prediction)
dt_renn_recall_score = recall_score(renn_train_labels_prepared, dt_renn_train_prediction)
print("precision of decision tree model on undersampling dataset is + %f" % dt_renn_precision_score)
print("recall of decision tree model on undersampling dataset is + %f" % dt_renn_recall_score)
print("-----------------------------------------")

#calculate the presion and recall of balanced sampling dataset
dt_smote_enn_precision_score = precision_score(smote_enn_train_labels_prepared, dt_smote_enn_train_prediction)
dt_smote_enn_recall_score = recall_score(smote_enn_train_labels_prepared, dt_smote_enn_train_prediction)
print("precision of decision tree model on balanced sampling dataset is + %f" % dt_smote_enn_precision_score)
print("recall of decision tree model on balanced sampling dataset is + %f" % dt_smote_enn_recall_score)

precision of decision tree model on oversampling dataset is + 0.902430
recall of decision tree model on oversampling dataset is + 1.000000
-----------------------------------------
precision of decision tree model on undersampling dataset is + 0.267206
recall of decision tree model on undersampling dataset is + 0.388235
-----------------------------------------
precision of decision tree model on balanced sampling dataset is + 0.903588
recall of decision tree model on balanced sampling dataset is + 0.952598


In [5]:
#train the model using K NEAREST NEIGHBOURS algorithm with 10 fold cross validation
from sklearn import neighbors
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score
n_neighbors = 10
knn_clf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
#train the model on the oversampling dataset
knn_ros_train_prediction = cross_val_predict(knn_clf.fit(ros_train_set_prepared, ros_train_labels_prepared), ros_train_set_prepared, ros_train_labels_prepared, cv=10)
#train the model on the undersampling dataset
knn_renn_train_prediction = cross_val_predict(knn_clf.fit(renn_train_set_prepared, renn_train_labels_prepared), renn_train_set_prepared, renn_train_labels_prepared, cv=10)
#train the model on the balanced sampling dataset
knn_smote_enn_train_prediction = cross_val_predict(knn_clf.fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared), smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10)

#calculate the presion and recall of oversampling dataset
knn_ros_precision_score = precision_score(ros_train_labels_prepared, knn_ros_train_prediction)
knn_ros_recall_score = recall_score(ros_train_labels_prepared, knn_ros_train_prediction)
print("precision of k-nearest neighbours model on oversampling dataset is + %f" % knn_ros_precision_score)
print("recall of k-nearest neighbours model on oversampling dataset is + %f" % knn_ros_recall_score)
print("-----------------------------------------")

#calculate the presion and recall of undersampling dataset
knn_renn_precision_score = precision_score(renn_train_labels_prepared, knn_renn_train_prediction)
knn_renn_recall_score = recall_score(renn_train_labels_prepared, knn_renn_train_prediction)
print("precision of k-nearest neighbours model on undersampling dataset is + %f" % knn_renn_precision_score)
print("recall of k-nearest neighbours model on undersampling dataset is + %f" % knn_renn_recall_score)
print("-----------------------------------------")

#calculate the presion and recall of balanced sampling dataset
knn_smote_enn_precision_score = precision_score(smote_enn_train_labels_prepared, knn_smote_enn_train_prediction)
knn_smote_enn_recall_score = recall_score(smote_enn_train_labels_prepared, knn_smote_enn_train_prediction)
print("precision of k-nearest neighbours model on balanced sampling dataset is + %f" % knn_smote_enn_precision_score)
print("recall of k-nearest neighbours model on balanced sampling dataset is + %f" % knn_smote_enn_recall_score)

precision of k-nearest neighbours model on oversampling dataset is + 0.776955
recall of k-nearest neighbours model on oversampling dataset is + 1.000000
-----------------------------------------
precision of k-nearest neighbours model on undersampling dataset is + 0.534091
recall of k-nearest neighbours model on undersampling dataset is + 0.276471
-----------------------------------------
precision of k-nearest neighbours model on balanced sampling dataset is + 0.893399
recall of k-nearest neighbours model on balanced sampling dataset is + 0.993163


In [6]:
#train the model using NAIVE BAYES algorithm with 10 fold cross validation
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score
gnb_clf = GaussianNB()
#train the model on the oversampling dataset
gnb_ros_train_prediction = cross_val_predict(gnb_clf.fit(ros_train_set_prepared, ros_train_labels_prepared), ros_train_set_prepared, ros_train_labels_prepared, cv=10)
#train the model on the undersampling dataset
gnb_renn_train_prediction = cross_val_predict(gnb_clf.fit(renn_train_set_prepared, renn_train_labels_prepared), renn_train_set_prepared, renn_train_labels_prepared, cv=10)
#train the model on the balanced sampling dataset
gnb_smote_enn_train_prediction = cross_val_predict(gnb_clf.fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared), smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10)

#calculate the presion and recall of oversampling dataset
gnb_ros_precision_score = precision_score(ros_train_labels_prepared, gnb_ros_train_prediction)
gnb_ros_recall_score = recall_score(ros_train_labels_prepared, gnb_ros_train_prediction)
print("precision of naive bayes model on oversampling dataset is + %f" % gnb_ros_precision_score)
print("recall of naive bayes model on oversampling dataset is + %f" % gnb_ros_recall_score)
print("-----------------------------------------")

#calculate the presion and recall of undersampling dataset
gnb_renn_precision_score = precision_score(renn_train_labels_prepared, gnb_renn_train_prediction)
gnb_renn_recall_score = recall_score(renn_train_labels_prepared, gnb_renn_train_prediction)
print("precision of naive bayes model on undersampling dataset is + %f" % gnb_renn_precision_score)
print("recall of naive bayes model on undersampling dataset is + %f" % gnb_renn_recall_score)
print("-----------------------------------------")

#calculate the presion and recall of balanced sampling dataset
gnb_smote_enn_precision_score = precision_score(smote_enn_train_labels_prepared, gnb_smote_enn_train_prediction)
gnb_smote_enn_recall_score = recall_score(smote_enn_train_labels_prepared, gnb_smote_enn_train_prediction)
print("precision of naive bayes model on balanced sampling dataset is + %f" % gnb_smote_enn_precision_score)
print("recall of naive bayes model on balanced sampling dataset is + %f" % gnb_smote_enn_recall_score)

precision of naive bayes model on oversampling dataset is + 0.593566
recall of naive bayes model on oversampling dataset is + 0.917150
-----------------------------------------
precision of naive bayes model on undersampling dataset is + 0.118022
recall of naive bayes model on undersampling dataset is + 0.870588
-----------------------------------------
precision of naive bayes model on balanced sampling dataset is + 0.698139
recall of naive bayes model on balanced sampling dataset is + 0.923428


In [7]:
#train the model using RULE BASED algorithm with 10 fold cross validation
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score
dc_clf = DummyClassifier(strategy='stratified')
#train the model on the oversampling dataset
dc_ros_train_prediction = cross_val_predict(dc_clf.fit(ros_train_set_prepared, ros_train_labels_prepared), ros_train_set_prepared, ros_train_labels_prepared, cv=10)
#train the model on the undersampling dataset
dc_renn_train_prediction = cross_val_predict(dc_clf.fit(renn_train_set_prepared, renn_train_labels_prepared), renn_train_set_prepared, renn_train_labels_prepared, cv=10)
#train the model on the balanced sampling dataset
dc_smote_enn_train_prediction = cross_val_predict(dc_clf.fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared), smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10)

#calculate the presion and recall of oversampling dataset
dc_ros_precision_score = precision_score(ros_train_labels_prepared, dc_ros_train_prediction)
dc_ros_recall_score = recall_score(ros_train_labels_prepared, dc_ros_train_prediction)
print("precision of rule based model on oversampling dataset is + %f" % dc_ros_precision_score)
print("recall of rule based model on oversampling dataset is + %f" % dc_ros_recall_score)
print("-----------------------------------------")

#calculate the presion and recall of undersampling dataset
dc_renn_precision_score = precision_score(renn_train_labels_prepared, dc_renn_train_prediction)
dc_renn_recall_score = recall_score(renn_train_labels_prepared, dc_renn_train_prediction)
print("precision of rule based model on undersampling dataset is + %f" % dc_renn_precision_score)
print("recall of rule based model on undersampling dataset is + %f" % dc_renn_recall_score)
print("-----------------------------------------")

#calculate the presion and recall of balanced sampling dataset
dc_smote_enn_precision_score = precision_score(smote_enn_train_labels_prepared, dc_smote_enn_train_prediction)
dc_smote_enn_recall_score = recall_score(smote_enn_train_labels_prepared, dc_smote_enn_train_prediction)
print("precision of rule based model on balanced sampling dataset is + %f" % dc_smote_enn_precision_score)
print("recall of rule based model on balanced sampling dataset is + %f" % dc_smote_enn_recall_score)

precision of rule based model on oversampling dataset is + 0.500840
recall of rule based model on oversampling dataset is + 0.494200
-----------------------------------------
precision of rule based model on undersampling dataset is + 0.066225
recall of rule based model on undersampling dataset is + 0.058824
-----------------------------------------
precision of rule based model on balanced sampling dataset is + 0.546373
recall of rule based model on balanced sampling dataset is + 0.542388


In [11]:
#the accuracies of the DECISION TREE model against each one of the ten folds when trained against the balanced sampling technique
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
score_method = make_scorer(accuracy_score)
print("The accuracies of the DECISION TREE model trained against the balanced sampling dataset is: ")
print(cross_val_score(dt_clf, smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the DECISION TREE model trained against the balanced sampling dataset is: 
[0.79353234 0.85074627 0.89303483 0.92288557 0.94264339 0.9276808
 0.94763092 0.91770574 0.95       0.945     ]


In [12]:
#the accuracies of the K NEAREST NEIGHBOURS model against each one of the ten folds when trained against the balanced sampling technique
print("The accuracies of the K NEAREST NEIGHBOURS model trained against the balanced sampling dataset is: ")
print(cross_val_score(knn_clf, smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the K NEAREST NEIGHBOURS model trained against the balanced sampling dataset is: 
[0.87064677 0.87064677 0.93283582 0.97761194 0.94014963 0.9276808
 0.97256858 0.93266833 0.9275     0.9625    ]


In [13]:
#the accuracies of the NAIVE BAYES model against each one of the ten folds when trained against the balanced sampling technique
print("The accuracies of the NAIVE BAYES model trained against the balanced sampling dataset is: ")
print(cross_val_score(gnb_clf, smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the NAIVE BAYES model trained against the balanced sampling dataset is: 
[0.75621891 0.71890547 0.79353234 0.87562189 0.71820449 0.73566085
 0.73566085 0.70573566 0.68       0.6775    ]


In [14]:
#the accuracies of the RULE BASED  model against each one of the ten folds when trained against the balanced sampling technique
print("The accuracies of the RULE BASED  model trained against the balanced sampling dataset is: ")
print(cross_val_score(dc_clf, smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the RULE BASED  model trained against the balanced sampling dataset is: 
[0.5        0.51741294 0.53731343 0.52487562 0.49625935 0.51620948
 0.51122195 0.50124688 0.52       0.495     ]


In [30]:
#apply two different feature selection techniques to the balanced sampling dataset
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
#tree-based feature selection
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared)
tb_model = SelectFromModel(clf, prefit=True)
tb_smote_enn_train_set_prepared = tb_model.transform(smote_enn_train_set_prepared)
print("Dataset with " + str(smote_enn_train_set_prepared.shape[1]) + " features is reduced to " + str(tb_smote_enn_train_set_prepared.shape[1])
      + " features after applying tree based feature selection technique")

#L1-based feature selection
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(smote_enn_train_set_prepared, smote_enn_train_labels_prepared)
l_model = SelectFromModel(lsvc, prefit=True)
l_smote_enn_train_set_prepared = l_model.transform(smote_enn_train_set_prepared)
print("Dataset with " + str(smote_enn_train_set_prepared.shape[1]) + " features is reduced to " + str(l_smote_enn_train_set_prepared.shape[1])
      + " features after applying L1-based feature selection technique")

Dataset with 24 features is reduced to 9 features after applying tree based feature selection technique
Dataset with 24 features is reduced to 8 features after applying L1-based feature selection technique


In [32]:
#the accuracies of the DECISION TREE model trained with balanced sampling dataset applying 2 feature selection technique
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
score_method = make_scorer(accuracy_score)
print("The accuracies of the DECISION TREE model (balanced sampling + tree based feature selection) is: ")
print(cross_val_score(dt_clf, tb_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the DECISION TREE model (balanced sampling + L1 based feature selection) is: ")
print(cross_val_score(dt_clf, l_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the DECISION TREE model (balanced sampling + tree based feature selection) is: 
[0.8681592  0.8159204  0.85820896 0.95522388 0.9276808  0.92269327
 0.94513716 0.88528678 0.9025     0.93      ]
The accuracies of the DECISION TREE model (balanced sampling + L1 based feature selection) is: 
[0.8159204  0.81840796 0.89054726 0.96268657 0.93017456 0.91521197
 0.93017456 0.89775561 0.935      0.925     ]


In [33]:
#the accuracies of the K NEAREST NEIGHBOURS model with balanced sampling dataset applying 2 feature selection technique
print("The accuracies of the K NEAREST NEIGHBOURS model (balanced sampling + tree based feature selection) is: ")
print(cross_val_score(knn_clf, tb_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the K NEAREST NEIGHBOURS model (balanced sampling + L1 based feature selection) is: ")
print(cross_val_score(knn_clf, l_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the K NEAREST NEIGHBOURS model (balanced sampling + tree based feature selection) is: 
[0.88557214 0.82338308 0.91044776 0.95024876 0.89027431 0.9201995
 0.93516209 0.90274314 0.8925     0.955     ]
The accuracies of the K NEAREST NEIGHBOURS model (balanced sampling + L1 based feature selection) is: 
[0.82587065 0.80845771 0.88308458 0.96268657 0.88279302 0.89027431
 0.94264339 0.90274314 0.925      0.95      ]


In [34]:
#the accuracies of the NAIVE BAYES model with balanced sampling dataset applying 2 feature selection technique
print("The accuracies of the NAIVE BAYES model (balanced sampling + tree based feature selection) is: ")
print(cross_val_score(gnb_clf, tb_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the NAIVE BAYES model (balanced sampling + L1 based feature selection) is: ")
print(cross_val_score(gnb_clf, l_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the NAIVE BAYES model (balanced sampling + tree based feature selection) is: 
[0.6641791  0.66666667 0.74875622 0.80597015 0.80049875 0.7605985
 0.78553616 0.80299252 0.74       0.7575    ]
The accuracies of the NAIVE BAYES model (balanced sampling + L1 based feature selection) is: 
[0.60447761 0.66169154 0.75870647 0.81343284 0.77306733 0.78802993
 0.74563591 0.74314214 0.7375     0.7275    ]


In [35]:
#the accuracies of the RULE BASED  model with balanced sampling dataset applying 2 feature selection technique
print("The accuracies of the RULE BASED  model (balanced sampling + tree based feature selection) is: ")
print(cross_val_score(dc_clf, tb_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))
print("The accuracies of the RULE BASED  model (balanced sampling + L1 based feature selection) is: ")
print(cross_val_score(dc_clf, l_smote_enn_train_set_prepared, smote_enn_train_labels_prepared, cv=10, scoring = score_method))

The accuracies of the RULE BASED  model (balanced sampling + tree based feature selection) is: 
[0.51492537 0.45273632 0.5199005  0.49253731 0.53366584 0.51620948
 0.4563591  0.50623441 0.5175     0.4375    ]
The accuracies of the RULE BASED  model (balanced sampling + L1 based feature selection) is: 
[0.50746269 0.51243781 0.51243781 0.47014925 0.49127182 0.49376559
 0.53117207 0.49376559 0.4775     0.525     ]
